In [ ]:
!pip install lxml

In [ ]:
!pip install bs4

In [1]:
from scrapy.spiders import SitemapSpider

In [3]:
from scrapy.spiders import CrawlSpider
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
import gc

In [4]:
gc.collect()

9

In [5]:
import nltk
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import numpy as np
import math 
from nltk.corpus import wordnet
import urllib.request
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib.request
import re
import requests as req
import pandas as pd
import numpy as np

import pymorphy2
nltk.download('stopwords');
english_stopwords = stopwords.words("english");
nltk.download('wordnet');
nltk.download('averaged_perceptron_tagger');
nltk.download('maxent_ne_chunker');
nltk.download('words');
lemmatizer = WordNetLemmatizer()
tokenizer = RegexpTokenizer(r'\w+')

[nltk_data] Downloading package stopwords to /home/alex/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/alex/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/alex/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/alex/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [6]:
import pandas as pd
import re
df_help = pd.read_excel("help_title.xlsx")
df_help = df_help[["Раздел", "url_1", "Подраздел", "url_2", "Статья", "url_3", "Абзац", "url_4"]]
df_help.head();

In [7]:
d_all_data = {}
d1 = df_help.set_index("url_4")["Абзац"].to_dict()
d2 = df_help.set_index("url_3")["Статья"].to_dict()
d3 = df_help.set_index("url_2")["Подраздел"].to_dict()
d4 = df_help.set_index("url_1")["Раздел"].to_dict()
d_all_data = {**d1,**d2,**d3,**d4}

In [8]:
d_all_data

{'https://help.wrike.com/hc/en-us/articles/209603709-Tasks#create': 'Create a Task',
 'https://help.wrike.com/hc/en-us/articles/209603709-Tasks#assign': 'Assign a Task',
 'https://help.wrike.com/hc/en-us/articles/209603709-Tasks#schedule': 'Schedule a Task',
 'https://help.wrike.com/hc/en-us/articles/209603709-Tasks#tag': 'Tag a Task (Organize Tasks into Folders)',
 'https://help.wrike.com/hc/en-us/articles/209603709-Tasks#follow': 'Follow a Task',
 'https://help.wrike.com/hc/en-us/articles/209603709-Tasks#unfollow': 'Unfollow a Task',
 'https://help.wrike.com/hc/en-us/articles/209603709-Tasks#print': 'Print a Task',
 'https://help.wrike.com/hc/en-us/articles/209603709-Tasks#depend': 'Add Dependencies to a Task',
 'https://help.wrike.com/hc/en-us/articles/209603709-Tasks#viewing': 'Viewing Tasks',
 'https://help.wrike.com/hc/en-us/articles/210323225-Task-View#overview': 'Overview',
 'https://help.wrike.com/hc/en-us/articles/210323225-Task-View#navigate': 'Navigate the Task View',
 'htt

In [9]:
len(d_all_data)

1391

In [10]:
import copy 

def preprocess_text(text_copy, tokenizer=RegexpTokenizer(r'\w+'), 
                    lemmatizer=WordNetLemmatizer(), stopwords=stopwords.words("english"), pure = False):
    text = copy.deepcopy(text_copy)
    tokens = tokenizer.tokenize(text)
    
    if pure: 
        tokens = [token.lower() for token in tokens]
        tokens = [token for token in tokens if token not in english_stopwords\
                  and token != " " \
                  and not token.isdigit()]
        lemmas = [lemmatizer.lemmatize(token) for token in tokens]
        clean_text = " ".join(lemmas)
    else: 
        clean_text = " ".join(tokens)
    return clean_text

In [11]:
link_ref = ['/hc/en-us/requests/new', 'https://help.wrike.com/hc/en-us/requests/new'
            ,'/hc/en-us/search?utf8=✓&query=Gantt+chart', 'https://help.wrike.com/hc/en-us/search?utf8=✓&query=Gantt+chart'
            ,'/hc/en-us/search?utf8=✓&query=Creating+tasks', 'https://help.wrike.com/hc/en-us/search?utf8=✓&query=Creating+tasks'
            ,'/hc/en-us/search?utf8=✓&query=Sharing+folders', 'https://help.wrike.com/hc/en-us/search?utf8=✓&query=Sharing+folders']


data_ref = "profill_names.csv"

def tag_visible(element):
    
    tag_ = element.parent.name
    tag_p = None
    try: 
        tag_p = element.parent.parent.name
    except: 
        pass
#     try: 
#         print(element.parent.text)
#     except: 
#         return False
    if tag_ in ['style', 'script', 'head', 'title', 'meta', '[document]', 'div', 'section', 'footer']:
        return False
    
    if tag_p in ['section', 'div']:
        return False
    
    if tag_p in ["p", "nav", "span"] and tag_ == "a":
        if element.parent.get_attribute_list("href")[-1] is not None  and \
               (not element.parent.get_attribute_list("href")[-1].startswith("/hc/en-us/signin")) and \
               (not element.parent.get_attribute_list("href")[-1].startswith("/hc/en-us/community")) and \
               (element.parent.get_attribute_list("href")[-1] not in link_ref) and \
               (element.parent.get_attribute_list("href")[-1].startswith("/hc/en-us/") or\
                element.parent.get_attribute_list("href")[-1][0] == "#"): 
            r1 = element.parent.get_attribute_list("href")[-1]
            if r1.startswith("/hc/en-us/profiles"):
                with open(data_ref, "a+") as f: 
                    f.write(re.sub('/hc/en-us/profiles/(\d*-)', '\n', r1))
            return True
            
        return False
    
    if isinstance(element, Comment):
        return False
    return True


def text_from_html(body):
    soup = BeautifulSoup(body, 'lxml')
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    text = u" ".join(t.strip() for t in visible_texts)
    x = re.sub('[\n\r]', '', text)
    x = re.sub(' +', ' ',text)
    if len(x) < 100: 
        return ''
    return  x


def href_from_html(body):
    soup = BeautifulSoup(body, 'lxml')
    s = ''
    for a in soup.find_all('a', href=True):
        site_data = a['href']
        if site_data in d_all_data or ('https://help.wrike.com' + site_data in d_all_data):  
            s += a['href'] + "," 
    return  s.split(",")

In [12]:
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy.crawler import CrawlerProcess

class TheFriendlyNeighbourhoodSpider(CrawlSpider):
    name = 'TheFriendlyNeighbourhoodSpider'
    allowed_domains = ['help.wrike.com']
    start_urls = ['https://help.wrike.com/hc/en-us/']
    custom_settings = {
    'CONCURRENT_REQUESTS': 10,
    'CONCURRENT_REQUESTS_PER_DOMAIN': 25,
    'ROBOTSTXT_OBEY': False,
    'CONCURRENT_ITEMS': 100,
    'REACTOR_THREADPOOL_MAXSIZE': 400,
    # Hides printing item dicts
    'LOG_LEVEL': 'INFO',
    'RETRY_ENABLED': False,
    'REDIRECT_MAX_TIMES': 1,
    # Grabs xpath before site finish loading
    'DOWNLOAD_FAIL_ON_DATALOSS': False,
    'DEPTH_PRIORITY': 1,
    'SCHEDULER_DISK_QUEUE' : 'scrapy.squeues.PickleFifoDiskQueue',
    'SCHEDULER_MEMORY_QUEUE' :'scrapy.squeues.FifoMemoryQueue'
    }
    rules = (
        Rule(LinkExtractor(), callback='parse_item', follow=True),
    )
    def parse_item(self, response):
        #print('Downloaded... ', response.url)
        if (not str(response.url).startswith("https://help.wrike.com/hc/en-us/signin")) and\
        ((str(response.url).startswith("https://help.wrike.com/hc/en-us")) \
                    or (str(response.url).startswith("https://www.wrike.com/blog/"))):
            data = response.url.split("/")[-1]
            if data == "": 
                data = response.url.split("/")[-2]
            filename = '/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/' + data + '.csv'
            filename_pure = '/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike_pure/' + data + '.csv'
            html = urllib.request.urlopen(response.url).read()
            f_text_raw = text_from_html(html)
            sites_data = href_from_html(html)
            intersect_set = set(sites_data) & set(d_all_data.keys())
            if str(response.url) in d_all_data or len(intersect_set): 
                print('Saving as :', filename, intersect_set)
                if len(f_text_raw) > 30: 
                    with open(filename_pure, 'w') as f_pure, open(filename, 'w') as f:
                        html = urllib.request.urlopen(response.url).read()
                        f_text_raw = text_from_html(html) 
                        f_text_pure = preprocess_text(f_text_raw, pure = True)
                        f_text = preprocess_text(f_text_raw)
                        print(filename)
                        f.write(str(str(response.url)+" \n"))
                        f.write(str({"intersect_setting": intersect_set})+" \n")
                        f.write(f_text)
                        f_pure.write(str(str(response.url)+" \n"))
                        f_pure.write(str({"intersect_setting": intersect_set})+" \n")
                        f_pure.write(f_text_pure)
                        


In [ ]:
%%time
if __name__ == "__main__":
    process = CrawlerProcess()
    process.crawl(TheFriendlyNeighbourhoodSpider)
    process.start()
    process.stop()

2020-09-13 03:04:18 [scrapy.utils.log] INFO: Scrapy 2.3.0 started (bot: scrapybot)
2020-09-13 03:04:18 [scrapy.utils.log] INFO: Versions: lxml 4.2.5.0, libxml2 2.9.8, cssselect 1.1.0, parsel 1.6.0, w3lib 1.22.0, Twisted 20.3.0, Python 3.7.1 (default, Dec 14 2018, 19:28:38) - [GCC 7.3.0], pyOpenSSL 18.0.0 (OpenSSL 1.1.1g  21 Apr 2020), cryptography 2.4.2, Platform Linux-5.4.0-45-generic-x86_64-with-debian-buster-sid
2020-09-13 03:04:18 [scrapy.utils.log] DEBUG: Using reactor: twisted.internet.epollreactor.EPollReactor
2020-09-13 03:04:18 [scrapy.crawler] INFO: Overridden settings:
{'CONCURRENT_REQUESTS': 10,
 'CONCURRENT_REQUESTS_PER_DOMAIN': 25,
 'DEPTH_PRIORITY': 1,
 'DOWNLOAD_FAIL_ON_DATALOSS': False,
 'LOG_LEVEL': 'INFO',
 'REACTOR_THREADPOOL_MAXSIZE': 400,
 'REDIRECT_MAX_TIMES': 1,
 'RETRY_ENABLED': False,
 'SCHEDULER_DISK_QUEUE': 'scrapy.squeues.PickleFifoDiskQueue',
 'SCHEDULER_MEMORY_QUEUE': 'scrapy.squeues.FifoMemoryQueue'}
2020-09-13 03:04:18 [scrapy.extensions.telnet] INFO: T

Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/search?utf8=%E2%9C%93&query=Sharing+folders.csv {'https://help.wrike.com/hc/en-us/sections/201834249-Folders-and-Projects', 'https://help.wrike.com/hc/en-us/categories/200942115-Tasks-Folders-Projects-and-Spaces'}
/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/search?utf8=%E2%9C%93&query=Sharing+folders.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/search?utf8=%E2%9C%93&query=Gantt+chart.csv {'https://help.wrike.com/hc/en-us/categories/200942125-Work-Views', 'https://help.wrike.com/hc/en-us/sections/201893075-Gantt-Chart'}
/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/search?utf8=%E2%9C%93&query=Gantt+chart.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/200942125-Work-Views.csv set()
/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/200942125-Work-Views.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/200942175-Security.c

2020-09-13 03:05:27 [scrapy.extensions.logstats] INFO: Crawled 128 pages (at 128 pages/min), scraped 0 items (at 0 items/min)


/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/search?page=6&query=Sharing+folders&utf8=%E2%9C%93.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/search?filter_by=knowledge_base&query=Gantt+chart&utf8=%E2%9C%93.csv {'https://help.wrike.com/hc/en-us/sections/201893035-Tasks', 'https://help.wrike.com/hc/en-us/categories/200942125-Work-Views', 'https://help.wrike.com/hc/en-us/sections/201893075-Gantt-Chart', 'https://help.wrike.com/hc/en-us/categories/200942115-Tasks-Folders-Projects-and-Spaces'}
/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/search?filter_by=knowledge_base&query=Gantt+chart&utf8=%E2%9C%93.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/201893075-Gantt-Chart.csv set()
/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/201893075-Gantt-Chart.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/210323545-Gantt-Chart-Snapshot.csv set()
/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wr

2020-09-13 03:06:24 [scrapy.extensions.logstats] INFO: Crawled 197 pages (at 69 pages/min), scraped 0 items (at 0 items/min)


...

Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/360014013293-Shortcut-to-open-task-in-new-tab.csv {'https://help.wrike.com/hc/en-us/articles/210322725-Keyboard-Shortcuts#available'}
/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/360014013293-Shortcut-to-open-task-in-new-tab.csv
Saving as : /home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/360014013293-Shortcut-to-open-task-in-new-tab?page=1.csv {'https://help.wrike.com/hc/en-us/articles/210322725-Keyboard-Shortcuts#available'}
/home/alex/chat_bot_wrike/wrike-bot/help_good_for_wrike/360014013293-Shortcut-to-open-task-in-new-tab?page=1.csv


2020-09-14 19:09:18 [scrapy.extensions.logstats] INFO: Crawled 1317751 pages (at 434 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 19:10:18 [scrapy.extensions.logstats] INFO: Crawled 1318276 pages (at 525 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 19:11:18 [scrapy.extensions.logstats] INFO: Crawled 1318647 pages (at 371 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 19:12:18 [scrapy.extensions.logstats] INFO: Crawled 1319237 pages (at 590 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 19:13:18 [scrapy.extensions.logstats] INFO: Crawled 1319794 pages (at 557 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 19:14:18 [scrapy.extensions.logstats] INFO: Crawled 1320257 pages (at 463 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 19:15:18 [scrapy.extensions.logstats] INFO: Crawled 1320837 pages (at 580 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 19:16:18 [scrapy.extensions.logstats] INFO: Crawled 1321387 pages (at 550 pages

2020-09-14 20:13:18 [scrapy.extensions.logstats] INFO: Crawled 1348348 pages (at 502 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 20:14:18 [scrapy.extensions.logstats] INFO: Crawled 1348704 pages (at 356 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 20:15:18 [scrapy.extensions.logstats] INFO: Crawled 1348962 pages (at 258 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 20:16:18 [scrapy.extensions.logstats] INFO: Crawled 1349441 pages (at 479 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 20:17:19 [scrapy.extensions.logstats] INFO: Crawled 1349939 pages (at 498 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 20:18:18 [scrapy.extensions.logstats] INFO: Crawled 1350380 pages (at 441 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 20:19:18 [scrapy.extensions.logstats] INFO: Crawled 1351023 pages (at 643 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 20:20:20 [scrapy.extensions.logstats] INFO: Crawled 1351550 pages (at 527 pages

2020-09-14 21:17:18 [scrapy.extensions.logstats] INFO: Crawled 1380842 pages (at 490 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 21:18:18 [scrapy.extensions.logstats] INFO: Crawled 1381353 pages (at 511 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 21:19:18 [scrapy.extensions.logstats] INFO: Crawled 1381696 pages (at 343 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 21:20:18 [scrapy.extensions.logstats] INFO: Crawled 1382170 pages (at 474 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 21:21:24 [scrapy.extensions.logstats] INFO: Crawled 1382519 pages (at 349 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 21:22:18 [scrapy.extensions.logstats] INFO: Crawled 1383017 pages (at 498 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 21:23:18 [scrapy.extensions.logstats] INFO: Crawled 1383605 pages (at 588 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 21:24:18 [scrapy.extensions.logstats] INFO: Crawled 1384209 pages (at 604 pages

2020-09-14 22:21:18 [scrapy.extensions.logstats] INFO: Crawled 1412468 pages (at 521 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 22:22:18 [scrapy.extensions.logstats] INFO: Crawled 1412858 pages (at 390 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 22:23:18 [scrapy.extensions.logstats] INFO: Crawled 1413365 pages (at 507 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 22:24:18 [scrapy.extensions.logstats] INFO: Crawled 1413870 pages (at 505 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 22:25:18 [scrapy.extensions.logstats] INFO: Crawled 1414420 pages (at 550 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 22:26:18 [scrapy.extensions.logstats] INFO: Crawled 1415015 pages (at 595 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 22:27:18 [scrapy.extensions.logstats] INFO: Crawled 1415555 pages (at 540 pages/min), scraped 0 items (at 0 items/min)
2020-09-14 22:28:19 [scrapy.extensions.logstats] INFO: Crawled 1416076 pages (at 521 pages